Installations of necessary libraries

In [1]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client 

Note: you may need to restart the kernel to use updated packages.


Import statements for including necessary packages

In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import BatchHttpRequest
from openai import OpenAI
from flask import Flask, request, jsonify
from threading import Thread
from dateutil import parser, tz
import json
import re
from datetime import datetime, timedelta, timezone
import random

Google calendar api integration

In [3]:
# Google Calendar API authentication
SCOPES = ["https://www.googleapis.com/auth/calendar"]
token_cache = {}
calendar_service_cache = {}


def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None


def get_google_credentials(email):
    """Retrieve Google credentials from cache or a token file."""
    if not is_valid_email(email):
        raise ValueError(f"Invalid email address: {email}")
    if email not in token_cache:
        token_path = f"Keys/{email.split('@')[0]}.token"
        try:
            token_cache[email] = Credentials.from_authorized_user_file(token_path, SCOPES)
        except FileNotFoundError:
            raise FileNotFoundError(f"Token file not found: {token_path}. Please ensure the token file is provided.")
        except Exception as e:
            raise Exception(f"Failed to load credentials for {email}: {str(e)}")
    return token_cache[email]


def get_google_calendar_service(email):
    """Build and cache a Google Calendar service object for a user."""
    if email not in calendar_service_cache:
        credentials = get_google_credentials(email)
        calendar_service_cache[email] = build("calendar", "v3", credentials=credentials)
    return calendar_service_cache[email]


def find_existing_calendar_event(calendar_service, request_id, attendees):
    """Check if a calendar event with the given Request_id already exists."""
    try:
        events_result = calendar_service.events().list(
            calendarId='primary',
            q=request_id,
            singleEvents=True
        ).execute()
        for event in events_result.get('items', []):
            event_attendees = [attendee['email'] for attendee in event.get('attendees', [])]
            if set(event_attendees).issuperset(set(attendees)):
                return event.get('id')
        return None
    except Exception as e:
        raise Exception(f"Error checking for existing event: {str(e)}")


def create_or_update_google_calendar_event(calendar_service, attendees, start_time, end_time, summary, location, request_id):
    """Create a new Google Calendar event or update an existing one."""
    event_body = {
        'summary': summary,
        'location': location,
        'description': f"Request_id: {request_id}",
        'start': {'dateTime': start_time, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': end_time, 'timeZone': 'Asia/Kolkata'},
        'attendees': [{'email': email} for email in attendees],
        'reminders': {'useDefault': True}
    }
    existing_event_id = find_existing_calendar_event(calendar_service, request_id, attendees)
    if existing_event_id:
        return calendar_service.events().update(
            calendarId='primary', eventId=existing_event_id, body=event_body
        ).execute()
    else:
        return calendar_service.events().insert(
            calendarId='primary', body=event_body
        ).execute()


def fetch_calendar_events_for_user(email, start_iso, end_iso):
    """Fetch calendar events for a specific user within a given time range."""
    events_list = []
    try:
        calendar_service = get_google_calendar_service(email)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start_iso,
            timeMax=end_iso,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        seen_events = set()
        for event in events:
            if not event.get("start") or not event.get("end"):
                continue
            
            start_time = event["start"].get("dateTime", event["start"].get("date") + "T00:00:00+05:30")
            end_time = event["end"].get("dateTime", event["end"].get("date") + "T00:00:00+05:30")

            if not start_time or not end_time:
                continue

            attendee_list = [attendee['email'] for attendee in event.get("attendees", [])] or ["SELF"]
            event_key = (start_time, end_time, event.get("summary", ""), tuple(sorted(attendee_list)))
            
            if event_key not in seen_events:
                seen_events.add(event_key)
                events_list.append({
                    "StartTime": start_time,
                    "EndTime": end_time,
                    "NumAttendees": len(set(attendee_list)),
                    "Attendees": list(set(attendee_list)),
                    "Summary": event.get("summary", "")
                })
    except Exception as e:
        return {"error": f"Error retrieving events for {email}: {str(e)}"}
    return events_list

Business logic to find the optimal meeting slots

In [4]:
def find_optimal_meeting_slot(calendar_service, attendees, search_date, duration_minutes, preferred_time='none'):
    """Find the best available time slot for all attendees and provide alternatives."""
    ist_tz = tz.gettz('Asia/Kolkata')
    working_start = search_date.replace(hour=9, minute=0, second=0, microsecond=0).astimezone(ist_tz)
    working_end = search_date.replace(hour=17, minute=0, second=0, microsecond=0).astimezone(ist_tz)
    time_min_utc = working_start.astimezone(tz.UTC).isoformat()
    time_max_utc = working_end.astimezone(tz.UTC).isoformat()

    body = {"timeMin": time_min_utc, "timeMax": time_max_utc, "items": [{"id": email} for email in attendees]}
    freebusy_result = calendar_service.freebusy().query(body=body).execute()

    attendee_conflicts = {email: len(cal.get('busy', [])) for email, cal in freebusy_result['calendars'].items()}

    all_busy_periods = []
    for cal in freebusy_result['calendars'].values():
        all_busy_periods.extend(cal.get('busy', []))
    
    # Merge overlapping busy periods
    sorted_busy = sorted(all_busy_periods, key=lambda x: x['start'])
    merged_busy = []
    if sorted_busy:
        merged_busy.append(sorted_busy[0])
        for period in sorted_busy[1:]:
            if merged_busy[-1]['end'] < period['start']:
                merged_busy.append(period)
            else:
                merged_busy[-1]['end'] = max(merged_busy[-1]['end'], period['end'])

    # Calculate free slots from merged busy periods
    free_slots = []
    current_time = parser.parse(time_min_utc)
    for busy_period in merged_busy:
        busy_start = parser.parse(busy_period['start'])
        if current_time < busy_start:
            free_slots.append({'start': current_time, 'end': busy_start})
        current_time = max(current_time, parser.parse(busy_period['end']))
    if current_time < parser.parse(time_max_utc):
        free_slots.append({'start': current_time, 'end': parser.parse(time_max_utc)})

    # Find all possible start times within the free slots
    duration_td = timedelta(minutes=int(duration_minutes))
    possible_start_times = []
    for slot in free_slots:
        if slot['end'] - slot['start'] >= duration_td:
            potential_start = slot['start']
            while potential_start + duration_td <= slot['end']:
                possible_start_times.append(potential_start)
                potential_start += timedelta(minutes=15) # Check in 15-minute increments

    if not possible_start_times:
        return None, None, [], attendee_conflicts, False

    # Logic to select the best slot and alternatives
    best_slot_start = None
    preferred_time_honored = False
    
    if preferred_time != 'none':
        # Determine preferred time window
        if preferred_time == 'morning':
            pref_start_hour, pref_end_hour = 9, 12
        elif preferred_time == 'afternoon':
            pref_start_hour, pref_end_hour = 13, 17
        else:
            try:
                pref_start_hour = int(preferred_time.split(':')[0])
                pref_end_hour = pref_start_hour + 1
            except (ValueError, IndexError):
                pref_start_hour, pref_end_hour = 9, 17 # Fallback
        
        for start_time in possible_start_times:
            if pref_start_hour <= start_time.astimezone(ist_tz).hour < pref_end_hour:
                best_slot_start = start_time
                preferred_time_honored = True
                break
    
    if best_slot_start is None:
        best_slot_start = possible_start_times[0] # Default to earliest

    # Collect alternative slots
    alternative_slots = []
    for alt_start in possible_start_times:
        if alt_start != best_slot_start and len(alternative_slots) < 3:
            alternative_slots.append({
                "start": alt_start.astimezone(ist_tz).isoformat(),
                "end": (alt_start + duration_td).astimezone(ist_tz).isoformat()
            })
    
    final_start_iso = best_slot_start.astimezone(ist_tz).isoformat()
    final_end_iso = (best_slot_start + duration_td).astimezone(ist_tz).isoformat()

    return final_start_iso, final_end_iso, alternative_slots, attendee_conflicts, preferred_time_honored

Setup of the vLLM using a Deepseek llm 

In [5]:
# vLLM and AI Agent setup
BASE_URL = "http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)


class SchedulingAIAgent:
    """An AI agent that handles meeting-related NLP tasks."""
    def __init__(self, client, model_path):
        self.client = client
        self.model_path = model_path

    def parse_meeting_request_from_email(self, email_content):
        """Extracts meeting details (participants, duration, etc.) from email text."""
        try:
            prompt = f"""
You are an AI assistant that extracts meeting details from emails. Parse the email and return a JSON object.

Examples:

Example 1:
Email: "Hi team, can we schedule a 30-minute sync meeting next Thursday afternoon to discuss the project?"
Output:
{{
    "participants": "team@amd.com",
    "meeting_duration": 30,
    "time_constraints": "next Thursday",
    "preferred_time": "afternoon"
}}

Example 2:
Email: "Let's meet tomorrow morning for 1 hour with John and Sarah to review the quarterly reports."
Output:
{{
    "participants": "john@amd.com, sarah@amd.com",
    "meeting_duration": 60,
    "time_constraints": "tomorrow",
    "preferred_time": "morning"
}}

Example 3:
Email: "Can we find 45 minutes on Monday at 2 PM for the client presentation with the marketing team?"
Output:
{{
    "participants": "marketing team@amd.com",
    "meeting_duration": 45,
    "time_constraints": "Monday",
    "preferred_time": "at 14:00"
}}

Example 4:
Email: "Hi all, we need to discuss the budget next week. Please find 90 minutes for everyone."
Output:
{{
    "participants": "all@amd.com",
    "meeting_duration": 90,
    "time_constraints": "next week",
    "preferred_time": "none"
}}

Instructions:
1. Extract participant names/references and append @amd.com
2. Extract duration in minutes (convert hours if needed)
3. Extract time constraints (specific days, relative terms like "tomorrow", "next week")
4. Extract time preferences (morning, afternoon, evening, or specific times like "at 14:00")
5. Return ONLY the JSON object, no additional text

Now parse this email:
---
{email_content}
---
"""
            response = self.client.chat.completions.create(
                model=self.model_path,
                temperature=0.0,
                messages=[{"role": "user", "content": prompt}]
            )
            parsed_json = json.loads(response.choices[0].message.content)
            
            # Validate the output from the LLM
            required_keys = ['participants', 'meeting_duration', 'time_constraints', 'preferred_time']
            if not all(key in parsed_json for key in required_keys):
                raise ValueError(f"LLM output missing required keys: {required_keys}")
            if not isinstance(parsed_json.get('meeting_duration'), (int, float)) or parsed_json['meeting_duration'] <= 0:
                raise ValueError("Invalid meeting duration from LLM")
            if not parsed_json.get('participants'):
                raise ValueError("No participants found by LLM")
                
            parsed_json['participants'] = [p.strip() for p in parsed_json['participants'].split(',')]
            return parsed_json
        except Exception as e:
            raise ValueError(f"LLM parsing failed: {str(e)}")

    def generate_slot_selection_reasoning(self, start_time, preferred_time, preferred_time_honored, attempted_days):
        """Generates a dynamic, natural language explanation for the scheduling decision."""
        try:
            prompt = f"""
            You are an AI assistant explaining a scheduling decision. Based on the following data, provide a concise, user-friendly "slot_selection_reasoning".

            Data:
            - Final Slot Day: {parser.parse(start_time).strftime('%A, %B %d')}
            - User Preference: '{preferred_time}'
            - Was Preference Met?: {preferred_time_honored}
            - Days Searched: {attempted_days}

            Instructions:
            - Create a 1-2 sentence explanation for why this slot was chosen.
            - If the preference was met, state that.
            - If the preference was not met, explain that this was the earliest available slot for all attendees.
            - If more than one day was searched, mention that the search was extended to find a suitable time.
            - Your tone should be helpful and professional. Return ONLY the string for the reasoning.
            """
            response = self.client.chat.completions.create(
                model=self.model_path,
                temperature=0.1,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content.strip()
        except Exception:
            # Fallback to a simple explanation if the LLM fails
            if preferred_time_honored:
                return f"This slot was chosen as it respects the '{preferred_time}' preference and is the earliest available option."
            else:
                return f"This is the earliest available time for all participants. The preference '{preferred_time}' could not be met."


ai_agent = SchedulingAIAgent(client, MODEL_PATH)
def process_meeting_request(request_data):
    """Main function to handle an incoming meeting request JSON."""
    num_api_calls = 0
    llm_start_time = datetime.now()
    try:
        # 1. Parse email content with the AI Agent (No change here)
        meeting_details = ai_agent.parse_meeting_request_from_email(request_data["EmailContent"])
        llm_time = (datetime.now() - llm_start_time).total_seconds()
        
        duration = meeting_details["meeting_duration"]
        time_constraint = meeting_details["time_constraints"].lower()
        preferred_time = meeting_details.get("preferred_time", "none")
        attendees = list(set([request_data["From"]] + [att["email"] for att in request_data["Attendees"] if is_valid_email(att["email"])]))
        if not attendees:
            raise ValueError("No valid email addresses provided for attendees")

        # 2. Determine the date range to search for a slot (WINNER'S LOGIC)
        # This section is now significantly smarter
        current_date = datetime.now(tz.gettz('Asia/Kolkata'))
        try:
            # Use the request's datetime as the primary reference for terms like "tomorrow" or "next week"
            reference_date = parser.parse(request_data["Datetime"]).astimezone(tz.gettz('Asia/Kolkata'))
        except (ValueError, TypeError):
            reference_date = current_date
        
        # Default to a 5-day search window if the LLM provides a generic constraint like "next week"
        start_date = reference_date
        end_date = start_date + timedelta(days=5) # Default search window

        # More precise date logic based on parsed constraints
        if "today" in time_constraint:
            start_date = reference_date
            end_date = reference_date
        elif "tomorrow" in time_constraint:
            start_date = reference_date + timedelta(days=1)
            end_date = start_date
        elif "next week" in time_constraint:
            days_until_monday = (7 - reference_date.weekday()) % 7
            start_date = reference_date + timedelta(days=days_until_monday)
            end_date = start_date + timedelta(days=4) # Monday to Friday
        else:
            # Logic for specific days like "monday", "tuesday", etc.
            weekday_map = {"monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, "friday": 4}
            day_name = next((day for day in weekday_map if day in time_constraint), None)
            if day_name:
                target_weekday = weekday_map[day_name]
                days_ahead = (target_weekday - reference_date.weekday() + 7) % 7
                if "next" in time_constraint or days_ahead == 0:
                    days_ahead += 7
                start_date = reference_date + timedelta(days=days_ahead)
                end_date = start_date

        # 3. Find an available slot by iterating through the determined date range (WINNER'S LOGIC)
        calendar_service = get_google_calendar_service(request_data['From'])
        num_api_calls += 1

        start_time, end_time, alternatives, conflicts, preferred_time_honored = None, None, [], {}, False
        found_slot = False
        
        # This loop is the core of the new logic
        current_search_day = start_date
        attempted_days_list = []
        while current_search_day <= end_date:
            # Only search on business days (Monday=0, Tuesday=1, ..., Friday=4)
            if current_search_day.weekday() < 5:
                attempted_days_list.append(current_search_day.strftime('%Y-%m-%d'))
                
                # Set the search time for the specific day
                search_date_for_slot = current_search_day.replace(hour=9, minute=0, second=0, microsecond=0)
                
                slot_start, slot_end, slot_alts, slot_conflicts, pref_honored = find_optimal_meeting_slot(
                    calendar_service, attendees, search_date_for_slot, duration, preferred_time
                )
                num_api_calls += 1

                if slot_start:
                    # Success! We found a slot.
                    start_time, end_time, alternatives, conflicts, preferred_time_honored = slot_start, slot_end, slot_alts, slot_conflicts, pref_honored
                    found_slot = True
                    break # Exit the loop immediately upon finding the first available slot
            
            current_search_day += timedelta(days=1)

        # 4. Generate Metadata and create event (No change here)
        slot_selection_reasoning = ""
        attempted_days = len(attempted_days_list)
        
        if not found_slot:
            # Fallback if no slot is found within the entire date range
            fallback_start = end_date.replace(hour=10, minute=30, second=0, microsecond=0)
            start_time = fallback_start.isoformat()
            end_time = (fallback_start + timedelta(minutes=int(duration))).isoformat()
            status_message = "warning: No free slot found in the requested range. A default time has been assigned as a placeholder."
            slot_selection_reasoning = f"Could not find a conflict-free slot for all attendees within the searched dates ({', '.join(attempted_days_list)}). This is a placeholder time."
            conflicts_resolved = 0
        else:
            # Create the event in the calendar
            create_or_update_google_calendar_event(calendar_service, attendees, start_time, end_time, request_data.get("Subject", "Meeting"), request_data.get("Location", ""), request_data["Request_id"])
            num_api_calls += 1
            status_message = "success: event created or updated"
            # Use the agent to generate the reasoning
            slot_selection_reasoning = ai_agent.generate_slot_selection_reasoning(start_time, preferred_time, preferred_time_honored, attempted_days)
            conflicts_resolved = 1

        # 5. Construct the final JSON output (No change here, except MetaData)
        processed_attendees = [{"email": email, "events": []} for email in attendees] # Placeholder
        
        output = {
            "Request_id": request_data["Request_id"], "Datetime": request_data["Datetime"],
            "Location": request_data.get("Location", ""), "From": request_data["From"],
            "Attendees": processed_attendees, "Subject": request_data.get("Subject", "Meeting"),
            "EmailContent": request_data["EmailContent"], "EventStart": start_time,
            "EventEnd": end_time, "Duration_mins": str(duration),
            "MetaData": {
                "status": status_message,
                "slot_selection_reasoning": slot_selection_reasoning,
                "attempted_days": attempted_days,
                "preferred_time_alloted": preferred_time_honored,
                "conflicts_resolved": conflicts_resolved,
                "alternative_slots": alternatives,
                "api_calls": num_api_calls,
                "llm_processing_time": round(llm_time, 6)
            }
        }
    except Exception as e:
        output = {
            "Request_id": request_data.get("Request_id", ""),
            "Subject": "Meeting Scheduling Failed",
            "EventStart": "", "EventEnd": "", "Duration_mins": "0",
            "MetaData": {
                "status": "error", "message": str(e),
                "llm_processing_time": (datetime.now() - llm_start_time).total_seconds() if 'llm_start_time' in locals() else 0
            }
        }
    return {"processed": True, "output": output}

Setup of the flask application


In [6]:
# Flask setup
app = Flask(__name__)

@app.route('/receive', methods=['POST'])
def handle_meeting_request():
    """Flask route to receive and process meeting requests."""
    try:
        request_data = request.get_json()
        print(f"\nReceived: {json.dumps(request_data, indent=2)}")
        response_data = process_meeting_request(request_data)
        print(f"\nSending:\n{json.dumps(response_data['output'], indent=2)}")
        return jsonify(response_data['output'])
    except Exception as e:
        error_response = {"processed": False, "output": {"status": "error", "message": str(e)}}
        print(f"\nError: {json.dumps(error_response, indent=2)}")
        return jsonify(error_response), 500


def start_flask_app():
    """Runs the Flask application, handling port conflicts."""
    try:
        app.run(host='0.0.0.0', port=5000)
    except OSError as e:
        if "Address already in use" in str(e):
            print("Port 5000 is already in use. Assuming server is running.")
        else:
            raise e

# Start the server in a background thread
flask_thread = Thread(target=start_flask_app, daemon=True)
if not flask_thread.is_alive():
    flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://134.199.197.49:5000
Press CTRL+C to quit



Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Client Feedback",
  "EmailContent": "Hi Team. We\u2019ve received the final feedback from the client. Let\u2019s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}


127.0.0.1 - - [20/Jul/2025 07:18:41] "POST /receive HTTP/1.1" 200 -



Sending:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": []
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": []
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": []
    }
  ],
  "Subject": "Client Feedback",
  "EmailContent": "Hi Team. We\u2019ve received the final feedback from the client. Let\u2019s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M.",
  "EventStart": "2025-07-23T10:30:00+05:30",
  "EventEnd": "2025-07-23T11:30:00+05:30",
  "Duration_mins": "60",
  "MetaData": {
    "status": "success: event created or updated",
    "slot_selection_reasoning": "The final slot chosen for the event is on Wednesday, July 23, at the user's preferred time of 'morning'. The preference was met, and the search was extended to find the 

Sample test requests

In [7]:
import requests
import json

url = "http://localhost:5000/receive"
headers = {"Content-Type": "application/json"}
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

{'Attendees': [{'email': 'userone.amd@gmail.com', 'events': []}, {'email': 'userthree.amd@gmail.com', 'events': []}, {'email': 'usertwo.amd@gmail.com', 'events': []}], 'Datetime': '19-07-2025T12:34:55', 'Duration_mins': '60', 'EmailContent': "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M.", 'EventEnd': '2025-07-23T11:30:00+05:30', 'EventStart': '2025-07-23T10:30:00+05:30', 'From': 'userone.amd@gmail.com', 'Location': 'IISc Bangalore', 'MetaData': {'alternative_slots': [{'end': '2025-07-23T11:45:00+05:30', 'start': '2025-07-23T10:45:00+05:30'}, {'end': '2025-07-23T12:00:00+05:30', 'start': '2025-07-23T11:00:00+05:30'}, {'end': '2025-07-23T12:15:00+05:30', 'start': '2025-07-23T11:15:00+05:30'}], 'api_calls': 3, 'attempted_days': 1, 'conflicts_resolved': 1, 'llm_processing_time': 0.372956, 'preferred_time_alloted': True, 'slot_selection_reasoning': "The final slot chosen for the event is on We

In [8]:
test_input ={
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Validation - Urgent",
    "EmailContent": "Hi Team. We’ve just received quick feedback from the client indicating that the instructions we provided aren’t working on their end. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM to discuss and resolve this issue."
}

result = process_meeting_request(test_input)
print(json.dumps(result['output'], indent=2))

{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": []
    },
    {
      "email": "userthree.amd@gmail.com",
      "events": []
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": []
    }
  ],
  "Subject": "Client Validation - Urgent",
  "EmailContent": "Hi Team. We\u2019ve just received quick feedback from the client indicating that the instructions we provided aren\u2019t working on their end. Let\u2019s prioritize resolving this promptly. Let\u2019s meet Monday at 9:00 AM to discuss and resolve this issue.",
  "EventStart": "2025-07-21T09:30:00+05:30",
  "EventEnd": "2025-07-21T10:00:00+05:30",
  "Duration_mins": "30",
  "MetaData": {
    "status": "success: event created or updated",
    "slot_selection_reasoning": "The final slot chosen for the event is on Monday, July 21, at 